In [2]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from scipy import stats
import math

In [3]:
# Read in the csv file to a dataframe
df = pd.read_csv('./city-data/fully_merged_data.csv')

In [4]:
# Drop columns that have barely any data or that may not be needed. (If needed just remove from the drop list)
df = df.drop(['incident_url', 'source_url', 'incident_url_fields_missing', 'gun_stolen', 'gun_type', 'location_description',
              'n_guns_involved', 'notes', 'participant_name', 'participant_relationship', 'sources', 'state_house_district', 'state_senate_district'], axis=1)
df

,Unnamed: 0,incident_id,date,state,city_or_county,address,n_killed,n_injured,congressional_district,incident_characteristics,...,participant_type,Unnamed: 0.1,geocode,population,poverty rate,income,age,property value,employees,wage
0,0,95289,2014-01-01,Michigan,Muskegon,300 block of Monroe Avenue,0,0,2.0,Shots Fired - No Injuries,...,0::Victim,1050.0,05000US26121,172148.0,165197.0,43920.0,39.0,101400.0,74691.0,34640.6
1,1,92307,2014-01-01,Wisconsin,Milwaukee,4700 block of N. 50th St,0,1,4.0,Shot - Wounded/Injured,...,0::Victim,2122.0,05000US55079,951448.0,930552.0,47607.0,34.7,151700.0,2835644.0,45566.0
2,2,92519,2014-01-01,Tennessee,Humboldt,NaN,0,0,8.0,Shots Fired - No Injuries,...,0::Subject-Suspect,NaN,NaN,652752.0,636034.0,38826.0,33.8,96800.0,2860897.0,43781.4
3,3,95244,2014-01-01,Virginia,Chesapeake,405 Shell Rd,0,0,4.0,Shots Fired - No Injuries,...,0::Victim,1999.0,05000US51550,233194.0,225837.0,69978.0,36.7,256500.0,4060169.0,56474.4
4,4,92251,2014-01-01,Ohio,Cleveland,12108 Marne Av,0,1,9.0,Shot - Wounded/Injured,...,0::Victim||1::Subject-Suspect,1485.0,06000US3903516000,145574.0,141161.0,54504.0,41.3,146400.0,72743.0,45369.7
5,5,217091,2014-01-01,Arizona,Phoenix,3400 E Sky Harbor Blvd,0,0,7.0,TSA Action,...,NaN,68.0,16000US0455000,1615041.0,1595700.0,52062.0,33.4,213300.0,2854089.0,46143.6
6,6,92512,2014-01-01,Florida,Riviera Beach,1141 West 31st Street,1,1,20.0,"Shot - Wounded/Injured||Shot - Dead (murder, a...",...,0::Victim||1::Victim||2::Subject-Suspect,NaN,NaN,132378.0,131231.0,46672.0,39.9,225400.0,8756756.0,44699.1
7,7,95236,2014-01-01,Texas,San Antonio,9200 block of West Alametos,0,1,20.0,Shot - Wounded/Injured,...,0::Victim,1921.0,16000US4865000,1492494.0,1469160.0,49268.0,33.5,133900.0,12204129.0,49308.7
8,8,92248,2014-01-01,Virginia,Russell County,Gravel Lick,1,0,9.0,"Shot - Dead (murder, accidental, suicide)",...,0::Victim||1::Subject-Suspect,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,165411,2014-01-01,Texas,Corpus Christi,4600 block of Valdez,0,1,27.0,Shot - Wounded/Injured||Domestic Violence,...,0::Victim||1::Subject-Suspect,1824.0,16000US4817000,325734.0,316503.0,54344.0,35.1,132400.0,12204129.0,49308.7


In [5]:
# Clean data here (just ideas):
# 1) Replace NaN's in particpant_age with average based off euclidean distance of lat and long and possibly population?
# 2) Then fill in participant_age_group accordingly?
# 3) Replace NaN's in participant_gender with both "unknown" and average of city/county to 
#    get perspective on whether data would be skewed. (Represents challenge for progress report)
# 4) Replace blanks in participant_status with "unknown"
# 5) Replace participant_type with "unknown"?

In [6]:
# Exploratory Analysis:

In [7]:
# Visual Analysis:

In [8]:
# 3) Replace NaN's in participant_gender with both "unknown" and average of city/county to 
#    get perspective on whether data would be skewed.

# Uncomment following row for unknown replacement
df['participant_gender'] = df['participant_gender'].replace(np.nan, "unknown")

In [9]:
pd.options.mode.chained_assignment = None  # default='warn'

def createGenderDictionary(df):
    cityGendersDict = {}
    
    # range is 4395 because that is number of rows
    for index in range(len(df)):
        
        # grab the current city/county which will end up being a key
        currentCity = df['city_or_county'][index]
        
        # if the current gender is known we will add this to the gender dictionary
        if(df['participant_gender'][index] != "unknown"):
            
            # if the current city isn't already a key make it one
            if(currentCity not in cityGendersDict):
                cityGendersDict[currentCity] = {"Male": 0, "Female": 0}
            
            # grab each gender for this row
            genders = df['participant_gender'][index].split("||")
            
            # update the current city's number of genders
            for gender in genders:
                if "Male" in gender:
                    cityGendersDict[currentCity]["Male"] += 1
                elif "Female" in gender:
                    cityGendersDict[currentCity]["Female"] += 1
                    
    # now just take the higher gender count (mode) and set that as the city's gender
    for key in cityGendersDict:
        if(cityGendersDict[key]["Male"] > cityGendersDict[key]["Female"]):
            cityGendersDict[key] = "0::Male"
        else:
            cityGendersDict[key] = "0::Female"
    
    return cityGendersDict


cityGendersDict = createGenderDictionary(df)

# loop here is to actually do the replacement into the dataframe
for index in df.index[df['participant_gender'] == "unknown"]:
    currentCity = df['city_or_county'][index]
    if currentCity in cityGendersDict:
        df['participant_gender'][index] = cityGendersDict[currentCity]
    
df

,Unnamed: 0,incident_id,date,state,city_or_county,address,n_killed,n_injured,congressional_district,incident_characteristics,...,participant_type,Unnamed: 0.1,geocode,population,poverty rate,income,age,property value,employees,wage
0,0,95289,2014-01-01,Michigan,Muskegon,300 block of Monroe Avenue,0,0,2.0,Shots Fired - No Injuries,...,0::Victim,1050.0,05000US26121,172148.0,165197.0,43920.0,39.0,101400.0,74691.0,34640.6
1,1,92307,2014-01-01,Wisconsin,Milwaukee,4700 block of N. 50th St,0,1,4.0,Shot - Wounded/Injured,...,0::Victim,2122.0,05000US55079,951448.0,930552.0,47607.0,34.7,151700.0,2835644.0,45566.0
2,2,92519,2014-01-01,Tennessee,Humboldt,NaN,0,0,8.0,Shots Fired - No Injuries,...,0::Subject-Suspect,NaN,NaN,652752.0,636034.0,38826.0,33.8,96800.0,2860897.0,43781.4
3,3,95244,2014-01-01,Virginia,Chesapeake,405 Shell Rd,0,0,4.0,Shots Fired - No Injuries,...,0::Victim,1999.0,05000US51550,233194.0,225837.0,69978.0,36.7,256500.0,4060169.0,56474.4
4,4,92251,2014-01-01,Ohio,Cleveland,12108 Marne Av,0,1,9.0,Shot - Wounded/Injured,...,0::Victim||1::Subject-Suspect,1485.0,06000US3903516000,145574.0,141161.0,54504.0,41.3,146400.0,72743.0,45369.7
5,5,217091,2014-01-01,Arizona,Phoenix,3400 E Sky Harbor Blvd,0,0,7.0,TSA Action,...,NaN,68.0,16000US0455000,1615041.0,1595700.0,52062.0,33.4,213300.0,2854089.0,46143.6
6,6,92512,2014-01-01,Florida,Riviera Beach,1141 West 31st Street,1,1,20.0,"Shot - Wounded/Injured||Shot - Dead (murder, a...",...,0::Victim||1::Victim||2::Subject-Suspect,NaN,NaN,132378.0,131231.0,46672.0,39.9,225400.0,8756756.0,44699.1
7,7,95236,2014-01-01,Texas,San Antonio,9200 block of West Alametos,0,1,20.0,Shot - Wounded/Injured,...,0::Victim,1921.0,16000US4865000,1492494.0,1469160.0,49268.0,33.5,133900.0,12204129.0,49308.7
8,8,92248,2014-01-01,Virginia,Russell County,Gravel Lick,1,0,9.0,"Shot - Dead (murder, accidental, suicide)",...,0::Victim||1::Subject-Suspect,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,165411,2014-01-01,Texas,Corpus Christi,4600 block of Valdez,0,1,27.0,Shot - Wounded/Injured||Domestic Violence,...,0::Victim||1::Subject-Suspect,1824.0,16000US4817000,325734.0,316503.0,54344.0,35.1,132400.0,12204129.0,49308.7


In [10]:
# Find whether or not a city has an abnormal amount of shootings based on average of all cities using the central 
# limit theorem.
# Possibly normalize data by having number of shootings be a proportion of total population?
# df1 = df.truncate(before=0, after=100000)

def createStateDict(df):
    stateDict = {}
    for index in range(len(df)):
        currentCity = df['city_or_county'][index]
        currentState = df['state'][index]
        
        # Creating a dictionary of dictionaries
        # Adds up the number of people killed in each city for each state
        if currentState not in stateDict:
            stateDict[currentState] = {}
        elif currentCity not in stateDict[currentState]:
            stateDict[currentState][currentCity] = df['n_killed'][index]
        else:
            stateDict[currentState][currentCity] += df['n_killed'][index]
    
    
    averageStateFatalities = {}
    
    # find average of each state
    for state in stateDict:
        total = 0
        numCities = 0
        average = 0
        for city in stateDict[state]:
            numCities += 1
            # replacing the total n_killed with the average for this city
            stateDict[state][city] = stateDict[state][city] / len(df[df.city_or_county == city])
            total += stateDict[state][city]
        if(numCities > 0):
            print(state, total, numCities)
            average = total / numCities
        
        
        averageStateFatalities[state] = round(average,2)
    
    print(averageStateFatalities)
    return averageStateFatalities, stateDict


avgStateFatalities, stateDict = createStateDict(df)

Michigan 125.656674029 549
Wisconsin 75.942821126 412
Tennessee 93.2099714443 440
Virginia 114.590983221 445
Ohio 165.836066417 734
Arizona 77.7695557752 168
Florida 208.15946353 738
Texas 381.168012373 871
New Jersey 68.4714744411 413
Pennsylvania 235.141455086 936
Massachusetts 16.6442281022 286
Maryland 89.3843548569 293
Nevada 19.3252755917 60
Colorado 66.4950845982 218
Illinois 130.351102965 577
Indiana 79.2623426443 433
Alaska 36.3437253551 145
Arkansas 86.697136026 319
Louisiana 118.618700617 385
North Carolina 184.02173266 641
Alabama 151.246412164 481
Connecticut 13.9563742829 152
Mississippi 116.314005907 374
California 340.141218107 967
Georgia 148.542224557 522
Kansas 43.0556601716 193
New York 106.62356948 858
Oklahoma 103.93991124 332
District of Columbia 7.88571210735 6
Kentucky 130.797646779 499
West Virginia 82.5066706134 368
Minnesota 75.1239377753 381
Missouri 135.174861365 503
Washington 84.4645719332 296
Iowa 37.8959258699 264
Nebraska 16.3330920376 117
South Carol

In [13]:
def determineAbnormalAmountOfShootings(avgStateFatalities, stateDict, state, city, numInstances):
    mu = avgStateFatalities[state]
    variance = 0
    numCities = 0
    print(len(stateDict[state]))
    
    # for loop to calculate the variance for this state
    for currentCity in stateDict[state]:
        numCities += 1
#         print(currentCity, stateDict[state][currentCity], avgStateFatalities[state])
        variance = variance + (stateDict[state][currentCity] - avgStateFatalities[state])**2
    variance = variance / (numCities - 1)
    
    # take a random sample from the dataframe with a length of numInstances
    # if numInstances exceeds total instances, then just use the total
    stateDF = df[df.city_or_county == city]
    try:
        sampleDF = stateDF.sample(n=numInstances)
    except ValueError:
        sampleDF = stateDF
    
    # take the mean of this random sample 
    yBar = sampleDF['n_killed'].mean()
    n = len(sampleDF)
    print(df[df.state == state])
#     print(sampleDF)
    U = math.sqrt(n)*(yBar-mu)/math.sqrt(variance)
    print(n, yBar, mu, variance, U)
    rv = stats.norm()
    print(rv.cdf(U))
    

determineAbnormalAmountOfShootings(avgStateFatalities, stateDict, "California", "San Francisco", 500)

967
        Unnamed: 0  incident_id        date       state    city_or_county  \
45              45        92162  2014-01-01  California       Los Angeles   
85              85        92386  2014-01-01  California       Bakersfield   
191            191        95232  2014-01-01  California          Glendale   
192            192        92389  2014-01-01  California            Fresno   
213            213        92393  2014-01-01  California            Merced   
216            216        92689  2014-01-02  California         San Diego   
234            234        92391  2014-01-02  California             Chico   
235            235        93346  2014-01-02  California             Vista   
297            297        92380  2014-01-02  California        Huntington   
301            301        92378  2014-01-02  California          San Jose   
317            317        93348  2014-01-02  California     National City   
346            346        92681  2014-01-03  California          San Jos